## Get biology project metadata

In [10]:
import re
import os
import json # Importar el módulo JSON

from bs4 import BeautifulSoup

In [11]:
def read_html_from_file(file_path: str) -> str | None:
    """
    Lee el contenido de un archivo HTML desde una ruta específica.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo '{file_path}': {e}")
        return None

def extract_project_data(row, pi_name, last_updated):
    """
    Extrae la información detallada de una única fila, incluyendo
    datos bibliográficos como DOI, PMID y título.
    """
    try:
        cells = row.find_all('td')
        
        # --- Datos del Proyecto ---
        project_cell = cells[1]
        project_link_tag = project_cell.find('a', href=re.compile("TASKID"))
        project_name = project_link_tag.get_text(strip=True)
        relative_link = project_link_tag['href']
        project_link = f"https://taskbook.nasaprs.com/tbp/{relative_link}"
        task_id_match = re.search(r'TASKID=(\d+)', relative_link)
        task_id = task_id_match.group(1) if task_id_match else "No encontrado"

        # --- Datos Bibliográficos (Celda más compleja) ---
        bibliography_cell = cells[2]
        cell_text = bibliography_cell.get_text(separator=' ', strip=True)

        # Título del Artículo (texto entre comillas)
        title_match = re.search(r'"([^"]+)"', cell_text)
        article_title = title_match.group(1).strip() if title_match else "No encontrado"

        # Autores (texto antes de la primera comilla)
        authors = cell_text.split('"')[0].strip()

        # Inicializar campos de enlaces e IDs
        doi_link = "No encontrado"
        pubmed_link = "No encontrado"
        pubmed_id = "No encontrado"
        pmc_link = "No encontrado"
        pmc_id = "No encontrado"

        # Buscar todos los enlaces <a> en la celda para extraer sus datos
        links_in_cell = bibliography_cell.find_all('a')
        for link in links_in_cell:
            href = link.get('href', '')
            text = link.get_text(strip=True)
            
            if "doi.org" in href:
                doi_link = href
            elif "db=pubmed" in href:
                pubmed_link = href
                if "PMID:" in text:
                    pubmed_id = text.replace("PMID:", "").strip()
            elif "pmc/articles" in href:
                pmc_link = href
                if "PMCID:" in text:
                    pmc_id = text.replace("PMCID:", "").strip()

        return {
            'PI Name': pi_name,
            'Bibliography Last Updated': last_updated,
            'Project Name': project_name,
            'Project Link': project_link,
            'Task ID': task_id,
            'Authors': authors,
            'Article Title': article_title,
            'DOI': doi_link,
            'PubMed ID': pubmed_id,
            'PubMed Link': pubmed_link,
            'PubMed Central ID': pmc_id,
            'PubMed Central Link': pmc_link
        }
    except (AttributeError, IndexError):
        return None
    
def parse_bibliography_results(html_content: str) -> list[dict]:
    """
    Parsea el contenido HTML de un ÚNICO archivo.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    results_table = soup.find('table', class_='intro')
    
    if not results_table:
        return []

    all_projects = []
    current_pi_name = None
    current_last_updated = None

    for row in results_table.find_all('tr'):
        if 'title' in row.get('class', []):
            cells = row.find_all('td')
            if len(cells) >= 5:
                current_pi_name = cells[2].get_text(strip=True)
                current_last_updated = cells[4].get_text(strip=True)
        
        elif row.find('a', href=re.compile("TASKID")):
            project_data = extract_project_data(row, current_pi_name, current_last_updated)
            if project_data:
                all_projects.append(project_data)
                
    return all_projects

def process_html_folder(folder_path: str) -> list[dict]:
    """
    Procesa todos los archivos HTML en una carpeta y agrega los resultados.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: La ruta '{folder_path}' no es un directorio válido.")
        return []
        
    aggregated_results = []
    print(f"Escaneando directorio: {folder_path}")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.html', '.htm')):
            full_path = os.path.join(folder_path, filename)
            print(f"  -> Procesando archivo: {filename}")
            
            html_content = read_html_from_file(full_path)
            if html_content:
                projects_from_file = parse_bibliography_results(html_content)
                for project in projects_from_file:
                    project['Source File'] = filename
                
                aggregated_results.extend(projects_from_file)

    return aggregated_results

def save_results_to_json(project_list: list[dict], output_path: str):
    """
    Guarda una lista de diccionarios en un archivo JSON.

    Args:
        project_list: La lista de datos de proyectos.
        output_path: La ruta del archivo JSON de salida.
    """
    if not project_list:
        print("No hay datos para guardar en el archivo JSON.")
        return
        
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            # json.dump escribe la lista en el archivo
            # indent=4 formatea el archivo para que sea legible
            # ensure_ascii=False permite guardar caracteres especiales como acentos
            json.dump(project_list, f, indent=4, ensure_ascii=False)
        print(f"\n¡Éxito! Los resultados se han guardado en: {output_path}")
    except Exception as e:
        print(f"\nError: No se pudo guardar el archivo JSON. Razón: {e}")

def display_results(project_list: list[dict]):
    """
    Imprime los resultados extraídos en la consola.
    """
    if not project_list:
        print("No se extrajeron datos para mostrar.")
        return

    print(f"\nTotal de registros extraídos: {len(project_list)}")


In [12]:

# --- Bloque principal de ejecución ---
if __name__ == "__main__":
    # --- MODIFICA ESTA LÍNEA ---
    # Pon la ruta a la CARPETA que contiene tus archivos HTML.
    folder_path = "taskbook/table_html"
    output_json_path = "taskbook/nasa_tasks.json"
    
    # 1. Procesar toda la carpeta de archivos HTML
    all_extracted_projects = process_html_folder(folder_path)
    
    # 2. Guardar los resultados combinados en un archivo JSON
    save_results_to_json(all_extracted_projects, output_json_path)
    
    # 3. Mostrar una vista previa de los resultados en la consola
    display_results(all_extracted_projects)

Escaneando directorio: taskbook/table_html
  -> Procesando archivo: 1.html
  -> Procesando archivo: 10.html
  -> Procesando archivo: 11.html
  -> Procesando archivo: 12.html
  -> Procesando archivo: 2.html
  -> Procesando archivo: 3.html
  -> Procesando archivo: 4.html
  -> Procesando archivo: 5.html
  -> Procesando archivo: 6.html
  -> Procesando archivo: 7.html
  -> Procesando archivo: 8.html
  -> Procesando archivo: 9.html

¡Éxito! Los resultados se han guardado en: taskbook/nasa_tasks.json

Total de registros extraídos: 557


## Donwload project html

In [1]:
import json
import os
import requests
from time import sleep

def download_project_htmls(json_file_path, output_dir="project_htmls"):
    """
    Lee un archivo JSON, extrae los enlaces de los proyectos y descarga
    el contenido HTML de cada enlace.

    Args:
        json_file_path (str): La ruta al archivo JSON de entrada.
        output_dir (str): El nombre del directorio donde se guardarán los archivos HTML.
    """
    # 1. Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Directorio creado: '{output_dir}'")

    # 2. Cargar los datos del archivo JSON
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            projects = json.load(f)
    except FileNotFoundError:
        print(f"Error: El archivo '{json_file_path}' no fue encontrado.")
        return
    except json.JSONDecodeError:
        print(f"Error: El archivo '{json_file_path}' no es un JSON válido.")
        return
    
    print(f"Se encontraron {len(projects)} proyectos en el archivo JSON.")

    # 3. Iterar sobre cada proyecto y descargar el HTML
    for i, project in enumerate(projects):
        # Usamos .get() para evitar errores si una clave no existe
        project_link = project.get("Project Link")
        task_id = project.get("Task ID")

        if not project_link or not task_id:
            print(f"Saltando proyecto #{i+1} por falta de 'Project Link' o 'Task ID'.")
            continue

        file_name = f"{task_id}.html"
        file_path = os.path.join(output_dir, file_name)

        print(f"Procesando proyecto {i+1}/{len(projects)} (Task ID: {task_id})...")

        # Si el archivo ya existe, lo saltamos para no descargarlo de nuevo
        if os.path.exists(file_path):
            print(f"  -> El archivo '{file_path}' ya existe. Saltando descarga.")
            continue

        try:
            # 4. Realizar la petición GET para obtener el HTML
            # Se añade un User-Agent para simular un navegador y evitar bloqueos
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            response = requests.get(project_link, headers=headers, timeout=15)
            
            # Lanza un error si la respuesta no fue exitosa (ej. 404, 500)
            response.raise_for_status()

            # 5. Guardar el contenido HTML en un archivo
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(response.text)
            
            print(f"  -> HTML descargado y guardado en '{file_path}'")

        except requests.exceptions.RequestException as e:
            print(f"  -> Error al descargar {project_link}: {e}")
        
        # Pequeña pausa para no saturar el servidor
        sleep(0.5)

    print("\nProceso completado.")



In [ ]:

# --- Punto de entrada del script ---
if __name__ == "__main__":
    # Nombre de tu archivo JSON. Asegúrate de que esté en la misma carpeta que el script.
    input_json_file = "taskbook/nasa_tasks.json" # <--- CAMBIA ESTE NOMBRE SI ES NECESARIO
    download_project_htmls(input_json_file)

## Get Jason data for project

In [ ]:
import os
import json
from bs4 import BeautifulSoup
import re

In [20]:
# ==============================================================================
# LA FUNCIÓN DE EXTRACCIÓN (LA MÁS ROBUSTA) SE MANTIENE IGUAL
# ==============================================================================

def clean_text(text):
    """Limpia el texto eliminando espacios extra y saltos de línea."""
    if not text:
        return ""
    return ' '.join(text.split())

def parse_project_details(html_content):
    """
    Analiza el contenido HTML de la página de un proyecto y extrae los detalles.
    Versión 3: Robusta, busca elementos por su texto visible en lugar de atributos JS.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    
    report_div = soup.find('div', id=lambda x: x and x.startswith('cf_layoutareaReport'))
    if not report_div:
        return {} # Devuelve vacío si no encuentra la estructura principal

    project_data = {}

    # --- 1. Información General del Proyecto ---
    title_span = report_div.find('span', class_='title')
    project_data['project_title'] = title_span.get_text(strip=True).replace('Project Title:', '').strip() if title_span else 'N/A'

    general_info_div = report_div.find('div', id=lambda x: x and x.startswith('pidetail'))
    if general_info_div:
        general_info_table = general_info_div.find('table')
        if general_info_table:
            bold_spans = general_info_table.find_all('span', class_='bold')
            for span in bold_spans:
                key = span.get_text(strip=True).replace(':', '').strip().replace(' ', '_').lower()
                value = span.next_sibling.strip() if span.next_sibling and isinstance(span.next_sibling, str) else ''
                project_data[key] = value

            discipline_element = general_info_table.find('b', string='Space Biology')
            if discipline_element and discipline_element.parent:
                project_data['research_discipline_element'] = clean_text(discipline_element.parent.get_text(strip=True))
            
            pdf_link = general_info_table.find('a', title='download in pdf')
            project_data['pdf_download_link'] = pdf_link['href'] if pdf_link else 'N/A'

    # --- 2. Investigador Principal (Principal Investigator) ---
    pi_data = {}
    pi_section_link = report_div.find('a', string=re.compile(r'Principal Investigator/Affiliation'))
    if pi_section_link:
        pi_name_raw = clean_text(pi_section_link.get_text(strip=True))
        pi_data['name'] = pi_name_raw.replace('Principal Investigator/Affiliation:', '').strip()
        pi_details_div = pi_section_link.find_next_sibling('div')
        if pi_details_div:
            address_td = pi_details_div.find('td', string='Address:&nbsp;')
            if address_td and address_td.find_next_sibling('td'):
                pi_data['address'] = clean_text(address_td.find_next_sibling('td').get_text(separator=' ', strip=True))
            
            bold_spans_pi = pi_details_div.find_all('span', class_='bold')
            for span in bold_spans_pi:
                key = span.get_text(strip=True).replace(':', '').strip().replace(' ', '_').lower()
                value = ''
                if span.find_next_sibling('a'):
                    value = span.find_next_sibling('a').get_text(strip=True)
                elif span.next_sibling and isinstance(span.next_sibling, str):
                    value = span.next_sibling.strip()
                pi_data[key] = value
    project_data['principal_investigator'] = pi_data

    # --- 3. Información del Proyecto (Detalles del Grant) ---
    project_info_data = {}
    grant_details_div = report_div.find('div', id=lambda x: x and x.startswith('grantdetail'))
    if grant_details_div:
        bold_spans_grant = grant_details_div.find_all('span', class_='bold')
        for span in bold_spans_grant:
            key = span.get_text(strip=True).replace(':', '').replace('.', '').replace('/', '').strip().replace(' ', '_').lower()
            value = ''
            value_node = span.next_sibling
            if value_node and isinstance(value_node, str):
                value = value_node.strip()
            if 'grant_monitor' in key and span.parent and span.parent.find('a'):
                email_link = span.parent.find('a')
                value = f"{value} ({email_link.get_text(strip=True)})"
            if value:
                project_info_data[key] = value

        title_cells = grant_details_div.find_all('td', class_='title')
        for cell in title_cells:
            key = cell.get_text(strip=True).replace(':', '').strip().replace(' ', '_').lower()
            value_cell = cell.find_next_sibling('td')
            if value_cell:
                value = value_cell.get_text('\n', strip=True)
                project_info_data[key] = value
    project_data['project_information'] = project_info_data

    # --- 4. Progreso y Bibliografía ---
    task_info_data = {}
    task_details_div = report_div.find('div', id=lambda x: x and x.startswith('taskdetail'))
    if task_details_div:
        progress_cell = task_details_div.find('td', string=re.compile(r'Task Progress:'))
        if progress_cell and progress_cell.find_next_sibling('td'):
            task_info_data['task_progress'] = progress_cell.find_next_sibling('td').get_text('\n', strip=True)

        bib_cell = task_details_div.find('td', string=re.compile(r'Bibliography:'))
        if bib_cell and bib_cell.find_next_sibling('td'):
            bib_info = bib_cell.find_next_sibling('td')
            description = bib_info.find(string=re.compile(r'Description:'))
            task_info_data['bibliography_description'] = clean_text(description) if description else 'N/A'
            articles_cell = bib_info.find('td', string=re.compile(r'Articles in Peer-reviewed Journals'))
            if articles_cell and articles_cell.find_next_sibling('td'):
                task_info_data['bibliography_articles'] = clean_text(articles_cell.find_next_sibling('td').get_text(strip=True))
            else:
                task_info_data['bibliography_articles'] = "Ver 'Show Cumulative Bibliography'"
    project_data['task_progress_and_bibliography'] = task_info_data

    return project_data


In [21]:

# ==============================================================================
# NUEVO BLOQUE PRINCIPAL PARA PROCESAR TODA LA CARPETA
# ==============================================================================

if __name__ == "__main__":
    # 1. Definir la carpeta de entrada y el archivo de salida
    input_folder = 'project_htmls'
    output_json_file = 'all_proyects.json'

    # 2. Preparar una lista para guardar los datos de todos los proyectos
    all_projects_data = []

    print(f"Iniciando el proceso de extracción desde la carpeta: '{input_folder}'...")

    # 3. Verificar si la carpeta de entrada existe
    if not os.path.isdir(input_folder):
        print(f"Error: La carpeta '{input_folder}' no fue encontrada. Asegúrate de que el nombre y la ruta son correctos.")
    else:
        # 4. Recorrer cada archivo en la carpeta de entrada
        for filename in os.listdir(input_folder):
            if filename.endswith('.html') or filename.endswith('.htm'):
                file_path = os.path.join(input_folder, filename)
                print(f"Procesando archivo: {filename}...")

                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        html = f.read()

                    # Extraer los datos usando nuestra función
                    extracted_data = parse_project_details(html)

                    # Si se extrajeron datos, añadirlos a la lista
                    if extracted_data:
                        # Buena práctica: añadir el nombre del archivo de origen a los datos
                        file_id = os.path.splitext(filename)[0]
                        # Añadir el nuevo campo 'id' al diccionario
                        extracted_data['id'] = file_id
                        extracted_data['source_file'] = filename
                        all_projects_data.append(extracted_data)
                        print(f" -> Éxito: Se extrajeron datos de {filename}")
                    else:
                        print(f" -> Advertencia: No se encontraron datos en {filename}")

                except Exception as e:
                    print(f" -> ERROR: Ocurrió un error inesperado al procesar {filename}: {e}")

        # 5. Guardar la lista completa en un único archivo JSON
        if all_projects_data:
            print(f"\nExtracción finalizada. Se procesaron {len(all_projects_data)} archivos con éxito.")
            print(f"Guardando todos los datos en '{output_json_file}'...")
            
            with open(output_json_file, 'w', encoding='utf-8') as f:
                json.dump(all_projects_data, f, ensure_ascii=False, indent=4)
            
            print("¡Proceso completado!")
        else:
            print("\nEl proceso finalizó, pero no se pudo extraer información de ningún archivo.")

Iniciando el proceso de extracción desde la carpeta: 'project_htmls'...
Procesando archivo: 10024.html...
 -> Éxito: Se extrajeron datos de 10024.html
Procesando archivo: 10062.html...
 -> Éxito: Se extrajeron datos de 10062.html
Procesando archivo: 10155.html...
 -> Éxito: Se extrajeron datos de 10155.html
Procesando archivo: 10174.html...
 -> Éxito: Se extrajeron datos de 10174.html
Procesando archivo: 10178.html...
 -> Éxito: Se extrajeron datos de 10178.html
Procesando archivo: 10181.html...
 -> Éxito: Se extrajeron datos de 10181.html
Procesando archivo: 10189.html...
 -> Éxito: Se extrajeron datos de 10189.html
Procesando archivo: 10190.html...
 -> Éxito: Se extrajeron datos de 10190.html
Procesando archivo: 10191.html...
 -> Éxito: Se extrajeron datos de 10191.html
Procesando archivo: 10196.html...
 -> Éxito: Se extrajeron datos de 10196.html
Procesando archivo: 10201.html...
 -> Éxito: Se extrajeron datos de 10201.html
Procesando archivo: 10204.html...
 -> Éxito: Se extrajeron 